In [ ]:
%load_ext autoreload
%autoreload 2

%env OMP_NUM_THREADS 1
%env MKL_NUM_THREADS 1

import sys
import os
import psutil
import pickle
import math
import argparse

# The deformation module library is not automatically installed yet, we need to add its path manually
sys.path.append("../../")

import numpy as np
import matplotlib.pyplot as plt
import torch

import implicitmodules.torch as dm

from joblib import Parallel, delayed
parallel = Parallel(n_jobs=psutil.cpu_count(logical=False))

torch.set_default_tensor_type(torch.FloatTensor)
torch.set_num_threads(1)

In [ ]:
aabb_source = dm.Utilities.AABB(-1., 1., -0.5, 0.5)
density = 8
length_x = int(density*aabb_source.width)
length_y = int(density*aabb_source.height)
mesh_pts_x, mesh_pts_y = torch.meshgrid([torch.linspace(aabb_source.xmin, aabb_source.xmax, length_x), torch.linspace(aabb_source.ymin, aabb_source.ymax, length_y)])
pts_implicit1 = dm.Utilities.grid2vec(mesh_pts_x, mesh_pts_y)
source = pts_implicit1.clone()
print(source.shape)

In [ ]:
# A polynomial model of order 3
def pol3(pos, abc):
    a = abc[0].unsqueeze(1)
    b = abc[1].unsqueeze(1)
    c = abc[2].unsqueeze(1)
    d = abc[3].unsqueeze(1)
    e = abc[4].unsqueeze(1)
    f = abc[5].unsqueeze(1)
    g = abc[6].unsqueeze(1)
    h = abc[7].unsqueeze(1)
    i = abc[8].unsqueeze(1)
    j = abc[9].unsqueeze(1)

    return a \
        + b*pos[:, 0] + c*pos[:, 1] \
        + d*pos[:, 0]**2 + e*pos[:, 1]**2 + f*pos[:, 0]*pos[:, 1] \
        + g*pos[:, 0]**3 + h*pos[:, 1]**3 + i*pos[:, 0]**2*pos[:, 1] + j*pos[:, 0]*pos[:, 1]**2

my_pol = pol3


In [ ]:
# The model parameters we use and would like to get back
abc_gt = torch.tensor([[1., 0.],
                       [4., 0.], [1., 1.],
                       [0., 1.], [0.5, 2.], [1., 1.],
                       [0., 0.1], [0.1, -0.2], [0., 0.5], [-0.2, -0.1]])
sigma_gt = 0.5
angle = 0.

def generate_target(pts, abc, sigma, angle):
    pts = torch.tensor(pts)
    abc = torch.tensor(abc)
    silent_dots_target = dm.DeformationModules.SilentLandmarks.build_from_points(pts)

    th = angle * math.pi * torch.ones(pts.shape[0])
    R_target = torch.stack([dm.Utilities.rot2d(t) for t in th])
    C_target = my_pol(pts, abc_gt).transpose(0, 1).unsqueeze(2)

    implicit1_target_pts_cotan = 3.*torch.rand_like(pts)
    implicit1_target_R_cotan = 3.*torch.rand_like(R_target)

    implicit1_target = dm.DeformationModules.ImplicitModule1(
        dm.Manifolds.Stiefel(2, pts.shape[0],
                             gd=(pts.view(-1).requires_grad_(),
                                 R_target.view(-1).requires_grad_()),
                             cotan=(implicit1_target_pts_cotan.clone().view(-1).requires_grad_(),
                                    implicit1_target_R_cotan.clone().view(-1).requires_grad_())),
        C_target, sigma_gt, 0.01, 1.)

    compound_target = dm.DeformationModules.CompoundModule([silent_dots_target,
                                                        implicit1_target])
    with torch.autograd.no_grad():
        dm.HamiltonianDynamic.shoot(dm.HamiltonianDynamic.Hamiltonian(compound_target), 20, "euler")

    return silent_dots_target.manifold.gd.view(-1, 2)

In [ ]:
# Callback function called when computing the energy of the model (before the actual shooting).
# Used to compute the C of the implicit module of order 1.
def parametric_c(init_manifold, modules, fitted_parameters):
    abc = fitted_parameters[-1]
    modules[1]._ImplicitModule1__C = my_pol(pts_implicit1, abc).transpose(0, 1).unsqueeze(2)


In [ ]:
sigma = sigma_gt
nu = 0.01
coeff = 1.

abc_init = torch.zeros(10, 2)
abc_init[0, 0] = 1.
abc_init[0, 1] = 1.
C_init = my_pol(pts_implicit1, abc_init).transpose(0, 1).unsqueeze(2)
C_target = my_pol(pts_implicit1, abc_gt).transpose(0, 1).unsqueeze(2)
th = 0. * math.pi * torch.ones(pts_implicit1.shape[0])
R = torch.stack([dm.Utilities.rot2d(t) for t in th])

In [ ]:
def set_correct_affinity():
    parser = argparse.ArgumentParser()
    parser.add_argument("--process-name")
    process = psutil.Process()
    args, _ = parser.parse_known_args(process.cmdline())
    worker_index = int(args.process_name.split('-')[1]) - 1
    process.cpu_affinity([worker_index])

In [ ]:
def compute_noisy_c_dot(noise_sigma, max_iter):
    set_correct_affinity()

    target = generate_target(pts_implicit1.numpy(), abc_gt, sigma_gt, 0.)
    target = target + noise_sigma*torch.randn_like(target)

    abc = abc_init.clone().requires_grad_()
    implicit1 = dm.DeformationModules.ImplicitModule1(
        dm.Manifolds.Stiefel(2, pts_implicit1.shape[0],
                             gd=(pts_implicit1.view(-1).requires_grad_(),
                                 R.view(-1).requires_grad_())),
        C_init, sigma_gt, nu, coeff)

    model = dm.Models.ModelPointsRegistration(
        [source],
        [implicit1],
        [dm.Attachment.EuclideanPointwiseDistanceAttachement(10.)],
        other_parameters=[abc],
        precompute_callback=parametric_c)

    fitter = dm.Models.ModelFittingScipy(model, 1., 1000.)
    fitter.fit([target], max_iter, log_interval=-1)

    C = model.modules[1].C.detach()
    return torch.dot((C/torch.norm(C)).contiguous().view(-1), (C_target/torch.norm(C_target)).contiguous().view(-1)).item()


In [ ]:
def compute_mean_noisy_c_dot(sigma, max_iter, loop_count):
    return sum([compute_noisy_c_dot(sigma, max_iter) for i in range(loop_count)])/loop_count
        

In [ ]:
noise_sigma = torch.linspace(0., 0.5, 10).tolist()
max_iter = 10
loops = 2

noisy_c_dot = parallel(delayed(compute_mean_noisy_c_dot)(ns, max_iter, loops) for ns in noise_sigma)

In [ ]:
plt.xlabel("$\sigma_\mathrm{noise}$")
plt.ylabel("$C_\mathrm{target} \cdot C_\mathrm{noisy}$")
plt.plot(noise_sigma, list(map(lambda x: np.abs(x), noisy_c_dot)))
plt.show()